In [1]:
import numpy as np
import csv

wine_path = '../data/p1ch4/tabular-wine/winequality-white.csv' #設定資料路徑
wineq_numpy = np.loadtxt(wine_path, dtype=np.float32, delimiter=";", skiprows=1) #delimiter 分隔符號，skiprows 跳過的列數

wineq_numpy

array([[ 7.  ,  0.27,  0.36, ...,  0.45,  8.8 ,  6.  ],
       [ 6.3 ,  0.3 ,  0.34, ...,  0.49,  9.5 ,  6.  ],
       [ 8.1 ,  0.28,  0.4 , ...,  0.44, 10.1 ,  6.  ],
       ...,
       [ 6.5 ,  0.24,  0.19, ...,  0.46,  9.4 ,  6.  ],
       [ 5.5 ,  0.29,  0.3 , ...,  0.38, 12.8 ,  7.  ],
       [ 6.  ,  0.21,  0.38, ...,  0.32, 11.8 ,  6.  ]], dtype=float32)

In [2]:
col_list = next(csv.reader(open(wine_path), delimiter=';')) #取得標題列所有特徵名稱

wineq_numpy.shape, col_list #輸出 wineq_numpy 陣列的 shape 及所有特徵名稱

((4898, 12),
 ['fixed acidity',
  'volatile acidity',
  'citric acid',
  'residual sugar',
  'chlorides',
  'free sulfur dioxide',
  'total sulfur dioxide',
  'density',
  'pH',
  'sulphates',
  'alcohol',
  'quality'])

In [3]:
import torch

wineq = torch.from_numpy(wineq_numpy)
wineq.shape, wineq.dtype

(torch.Size([4898, 12]), torch.float32)

In [4]:
data = wineq[:, :-1] #選擇除了最後一行(品質分數)以外的所有資料
data, data.shape

(tensor([[ 7.0000,  0.2700,  0.3600,  ...,  3.0000,  0.4500,  8.8000],
         [ 6.3000,  0.3000,  0.3400,  ...,  3.3000,  0.4900,  9.5000],
         [ 8.1000,  0.2800,  0.4000,  ...,  3.2600,  0.4400, 10.1000],
         ...,
         [ 6.5000,  0.2400,  0.1900,  ...,  2.9900,  0.4600,  9.4000],
         [ 5.5000,  0.2900,  0.3000,  ...,  3.3400,  0.3800, 12.8000],
         [ 6.0000,  0.2100,  0.3800,  ...,  3.2600,  0.3200, 11.8000]]),
 torch.Size([4898, 11]))

In [5]:
target = wineq[:, -1] #取出 wineq 中的品質分數，做為目標張量
target, target.shape

(tensor([6., 6., 6.,  ..., 6., 7., 6.]), torch.Size([4898]))

In [6]:
target = wineq[:, -1].long() #將 wineq 中的品質分數轉為整數
target

tensor([6, 6, 6,  ..., 6, 7, 6])

In [7]:
target_onehot = torch.zeros(target.shape[0], 10) #建立 shape 為 [4980, 10]，值均為 0 的張量，用來存放編碼結果

a = target_onehot.scatter_(1, target.unsqueeze(1), 1.0) #依 targe 的內容進行編碼

#scatter_ 有 3 個參數
# 1. 要沿著哪個軸進行 one-hot 編碼( 1代表第 1 軸)
# 2. 要進行 one-hot 編碼的來源張量(張量內的值表示 target_onehot 中哪一個索引位置要被設為非 0 的值)
# 3. 設定 one-hot 編碼中非 0 的值為多少，通常是 1
# 總結: 對於每一列，取目標標籤(就是第 2 個參數所指定的張量，在以上例子中，即個別樣本的品質分數)的值作為 one-hot 向量中的索引，
# 設置相應值為 1.0
# !! 來源張量(第二個參數)的軸數，必須與我們散佈(scatter)到的結果張量(target_onehot)相同，
# 由於 target_onehot有兩個軸(4898x10)因此要用 unsqueeze() 將 target(1軸向量)增加 1軸
a, a.shape

(tensor([[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 1., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]),
 torch.Size([4898, 10]))

In [8]:
target_unsqueezed = target.unsqueeze(1) #添加一個軸至第 1 軸
print(target_unsqueezed)
print(target_unsqueezed.shape)

tensor([[6],
        [6],
        [6],
        ...,
        [6],
        [7],
        [6]])
torch.Size([4898, 1])


In [10]:
data_mean = torch.mean(data, dim=0) #計算平均值，dim=0表示指定要沿著第 0 軸做計算

data_mean #個化學性質的平均值

tensor([6.8548e+00, 2.7824e-01, 3.3419e-01, 6.3914e+00, 4.5772e-02, 3.5308e+01,
        1.3836e+02, 9.9403e-01, 3.1883e+00, 4.8985e-01, 1.0514e+01])

In [11]:
data_var = torch.var(data, dim=0) #計算標準差
data_var

tensor([7.1211e-01, 1.0160e-02, 1.4646e-02, 2.5726e+01, 4.7733e-04, 2.8924e+02,
        1.8061e+03, 8.9455e-06, 2.2801e-02, 1.3025e-02, 1.5144e+00])

In [12]:
data_normalized = (data - data_mean) / torch.sqrt(data_var) #用 Python 實現標準化的公式 
data_normalized

tensor([[ 1.7208e-01, -8.1761e-02,  2.1326e-01,  ..., -1.2468e+00,
         -3.4915e-01, -1.3930e+00],
        [-6.5743e-01,  2.1587e-01,  4.7996e-02,  ...,  7.3995e-01,
          1.3422e-03, -8.2419e-01],
        [ 1.4756e+00,  1.7450e-02,  5.4378e-01,  ...,  4.7505e-01,
         -4.3677e-01, -3.3663e-01],
        ...,
        [-4.2043e-01, -3.7940e-01, -1.1915e+00,  ..., -1.3130e+00,
         -2.6153e-01, -9.0545e-01],
        [-1.6054e+00,  1.1666e-01, -2.8253e-01,  ...,  1.0049e+00,
         -9.6251e-01,  1.8574e+00],
        [-1.0129e+00, -6.7703e-01,  3.7852e-01,  ...,  4.7505e-01,
         -1.4882e+00,  1.0448e+00]])

In [13]:
bad_indexes = target <= 3
bad_indexes.shape, bad_indexes.dtype, bad_indexes.sum()

(torch.Size([4898]), torch.bool, tensor(20))

In [14]:
bad_data = data[bad_indexes] #利用索引篩選出符合目標的項目
bad_data.shape

torch.Size([20, 11])

In [18]:
bad_data = data[target <= 3] #篩選出品質 差 的項目

#代表邏輯運算中的 and，即要前後條件都符合，才會傳回 true
mid_data = data[(target > 3) & (target < 7)] #篩選出品質 中等 的項目
good_data = data[target >= 7] #篩選出品質 好 的項目

bad_mean = torch.mean(bad_data, dim=0) #計算品質 差 的項目中，各化學性質的平均值
mid_mean = torch.mean(mid_data, dim=0) #計算品質 中等 的項目中，各化學性質的平均值
good_mean = torch.mean(good_data, dim=0) #計算品質 好 的項目中，各化學性質的平均值

for i, args in enumerate(zip(col_list, bad_mean, mid_mean, good_mean)):
    print('{:2} {:20} {:6.2f} {:6.2f} {:6.2f}'.format(i, *args)) #印出結果

 0 fixed acidity          7.60   6.89   6.73
 1 volatile acidity       0.33   0.28   0.27
 2 citric acid            0.34   0.34   0.33
 3 residual sugar         6.39   6.71   5.26
 4 chlorides              0.05   0.05   0.04
 5 free sulfur dioxide   53.33  35.42  34.55
 6 total sulfur dioxide 170.60 141.83 125.25
 7 density                0.99   0.99   0.99
 8 pH                     3.19   3.18   3.22
 9 sulphates              0.47   0.49   0.50
10 alcohol               10.34  10.26  11.42


In [21]:
total_sulfur_threshold = 141.83 #設定閾值為 141.83，若超出此數字則為品質差的酒，低於此數字則為好酒
total_sulfur_data = data[:, 6] #取出資料集的總二氧化硫量資訊

predicted_indexes = torch.lt(total_sulfur_data, total_sulfur_threshold) #lt 可以挑出 A 中，值 < B 的項目索引

predicted_indexes.shape, predicted_indexes.dtype, predicted_indexes.sum() #輸出小於閾值的總樣本數(即預測的好酒數量)

(torch.Size([4898]), torch.bool, tensor(2727))

In [22]:
actual_indexes = target > 5
actual_indexes.shape, actual_indexes.dtype, actual_indexes.sum()

(torch.Size([4898]), torch.bool, tensor(3258))

In [23]:
#actual_indexes 實際為好酒
#predicted_indexes 預測為好酒
#利用 & 找出皆為 True 的項目索引
n_matches = torch.sum(actual_indexes & predicted_indexes).item() #計算結果吻合的項目數量

n_predicted = torch.sum(predicted_indexes).item() #預測為好酒的項目數量
n_actual = torch.sum(actual_indexes).item() #實際為好酒的項目數量
n_matches, n_matches/n_predicted, n_matches/n_actual

(2018, 0.74000733406674, 0.6193984039287906)